<a href="https://colab.research.google.com/github/wksmirnowa/machinelearning_homeworks/blob/master/HW03_crossvalidation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import time
import datetime
import numpy as np
import pandas as pd
pd.set_option('max_columns', None)
import seaborn
import plotly.offline as py
py.init_notebook_mode(connected=True)
import warnings
warnings.filterwarnings('ignore')
from pprint import pprint

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [0]:
from scipy import stats
from scipy.sparse import csr_matrix
from sklearn import preprocessing
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold, cross_val_score, GridSearchCV, RepeatedStratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn import model_selection
from sklearn.metrics import *
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import fetch_20newsgroups #импортируем корпус 20newsgroups
from sklearn.datasets import make_classification
from sklearn.feature_extraction.text import CountVectorizer


In [3]:
!pip install eli5

In [3]:
import eli5

Using TensorFlow backend.


In [0]:
# # Если нужен доступ к данным из kaggle
# !pip install -q kaggle
# !mkdir -p ~/.kaggle
# !cp kaggle.json ~/.kaggle/
# !ls ~/.kaggle
# !chmod 600 /root/.kaggle/kaggle.json

In [0]:
# !kaggle kernels pull artgor/how-to-not-overfit

Посмотрим на данные

In [0]:
newsgroups_train = fetch_20newsgroups(subset='train')

In [0]:
pprint(list(newsgroups_train.target_names))

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']


Выберем 4 темы

In [0]:
categories = ['sci.med', 'sci.space', 'sci.electronics', 'comp.graphics']
data_train = fetch_20newsgroups(subset='train', categories=categories, shuffle=True, random_state=42)
data_test = fetch_20newsgroups(subset='test', categories=categories, shuffle=True, random_state=42)

In [6]:
len(data_train.data), len(data_test.data)

(2362, 1572)

Векторизуем с помощью CountVectorizer, обучим несколько моделей сначала без кросс-валидации и оценим с помощью macro_f1. Также добавим функцию results для проверки на тесте и трейне. 

In [0]:
count_vect = CountVectorizer()
y_train = data_train.target
y_test = data_test.target
X_train = count_vect.fit_transform(data_train.data) 
X_test = count_vect.transform(data_test.data) 

In [0]:
def show_macro_f1(true, predicted):
  macro_f1 = f1_score(true, predicted, average = 'macro')
  print('macro F1={0:1.4f}'.format(macro_f1))

In [0]:
def results(model=None, X_train=X_train, X_test=X_test):
  model_fit = model.fit(X_train, y_train)
  train_preds = model_fit.predict_proba(X_train)[:, 1]
  test_preds = model_fit.predict_proba(X_test)[:, 1]
  print('result on train: {}'.format(f1_score(y_train, np.round(train_preds), average = 'macro')))
  print('result on test: {}'.format(f1_score(y_test, np.round(test_preds), average = 'macro')))

## Кросс-валидация

In [0]:
n_fold = 10
folds = KFold(n_splits=n_fold, shuffle=True, random_state=0)
stratified_folds = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=0)
repeated_folds = RepeatedStratifiedKFold(n_splits=n_fold, n_repeats=20, random_state=0)

In [0]:
def make_grid_search(model=None, params_grid={'': []}, scoring='', n_jobs=0, folds=folds):
  grid_search = GridSearchCV(model, param_grid=params_grid, cv=folds, scoring=scoring, n_jobs=n_jobs)
  grid_search.fit(X_train, y_train)
  print(f'Best score: {grid_search.best_score_}')
  print(f'Best parameters: {grid_search.best_params_}')


## eli5

In [0]:
def get_word_by_index(feature):

  index_to_word = {v:k for k, v in count_vect.vocabulary_.items()}
  feature = int(feature.strip('x'))
  return index_to_word[feature]

def analyze_features(model=None, n=10, categories=categories):

  if model == model_DT or model == best_model_DT:
    explain_weights = eli5.formatters.as_dataframe.explain_decision_tree(model)

  explain_weights = eli5.formatters.as_dataframe.explain_weights_df(model)

  if 'target' not in explain_weights.columns:
    explain_weights['word'] = explain_weights.feature.apply(get_word_by_index)
    explain_weights = explain_weights.nlargest(n, 'weight')
    explain_weights.reset_index(drop=True, inplace=True)

    return explain_weights

  target_features = []

  for target in explain_weights.target.unique():
    features = explain_weights.loc[explain_weights.target == target]
    features = features.nlargest(n, 'weight')
    features = features.loc[features.feature != '<BIAS>']
    features['word'] = features.feature.apply(get_word_by_index)
    features['category'] = categories[target]
    target_features.append(features)

  table = pd.concat(target_features, axis=0)
  table.reset_index(drop=True, inplace=True)

  return table

## Логистическая регрессия

До применения кросс-валидации.

In [0]:
model_LR = LogisticRegression().fit(X_train, y_train)
predicted_LR = model_LR.predict(X_test)

In [113]:
show_macro_f1(y_test, predicted_LR)

macro F1=0.8643


In [114]:
results(model_LR)

result on train: 0.37399150743099785
result on test: 0.32768927596034947


После

In [0]:
%%time

LR = linear_model.LogisticRegression()
parameters_LR = {'class_weight' : ['balanced', None],
                  'penalty' : ['l2', 'l1'],
                  'solver' : ['liblinear', 'saga'],
                  'C' : [0.15, 1.0, 10.0, 100.0],
                  'max_iter': [10,50,100]
                 }

make_grid_search(LR, parameters_LR, 'f1_macro')

Fitting 5 folds for each of 96 candidates, totalling 480 fits
[CV] C=0.15, class_weight=balanced, max_iter=10, penalty=l2, solver=liblinear 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  C=0.15, class_weight=balanced, max_iter=10, penalty=l2, solver=liblinear, score=0.945, total=   0.5s
[CV] C=0.15, class_weight=balanced, max_iter=10, penalty=l2, solver=liblinear 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.5s remaining:    0.0s


[CV]  C=0.15, class_weight=balanced, max_iter=10, penalty=l2, solver=liblinear, score=0.932, total=   0.5s
[CV] C=0.15, class_weight=balanced, max_iter=10, penalty=l2, solver=liblinear 


...

[CV]  C=0.15, class_weight=balanced, max_iter=10, penalty=l2, solver=liblinear, score=0.928, total=   0.6s
[CV] C=0.15, class_weight=balanced, max_iter=10, penalty=l2, solver=liblinear 


[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    2.2s remaining:    0.0s


[CV]  C=0.15, class_weight=balanced, max_iter=10, penalty=l2, solver=liblinear, score=0.939, total=   0.5s
[CV] C=0.15, class_weight=balanced, max_iter=10, penalty=l2, solver=saga 
[CV]  C=0.15, class_weight=balanced, max_iter=10, penalty=l2, solver=saga, score=0.539, total=   0.2s
[CV] C=0.15, class_weight=balanced, max_iter=10, penalty=l2, solver=saga 
[CV]  C=0.15, class_weight=balanced, max_iter=10, penalty=l2, solver=saga, score=0.467, total=   0.1s
[CV] C=0.15, class_weight=balanced, max_iter=10, penalty=l2, solver=saga 
[CV]  C=0.15, class_weight=balanced, max_iter=10, penalty=l2, solver=saga, score=0.506, total=   0.1s
[CV] C=0.15, class_weight=balanced, max_iter=10, penalty=l2, solver=saga 
[CV]  C=0.15, class_weight=balanced, max_iter=10, penalty=l2, solver=saga, score=0.506, total=   0.1s
[CV] C=0.15, class_weight=balanced, max_iter=10, penalty=l2, solver=saga 
[CV]  C=0.15, class_weight=balanced, max_iter=10, penalty=l2, solver=saga, score=0.496, total=   0.1s
[CV] C=0.15, 

[Parallel(n_jobs=1)]: Done 480 out of 480 | elapsed: 131.2min finished


Best score: 0.9373445730461892
Best parameters: {'C': 100.0, 'class_weight': None, 'max_iter': 100, 'penalty': 'l1', 'solver': 'liblinear'}
CPU times: user 2h 12min 36s, sys: 1min 15s, total: 2h 13min 51s
Wall time: 2h 12min 33s


Попробуем применить эти параметры и снова обучить модель

In [218]:
%%time
best_model_LR = LogisticRegression(penalty='l1', C=100.0, solver='liblinear', max_iter=100).fit(X_train, y_train)
predicted_LR = best_model_LR.predict(X_test)

CPU times: user 1min 13s, sys: 9.55 ms, total: 1min 13s
Wall time: 1min 13s


In [219]:
show_macro_f1(y_test, predicted_LR)

macro F1=0.8775


In [220]:
results(best_model_LR)

result on train: 0.37399150743099785
result on test: 0.33004405157827177


У логистической регрессии f1 макро было macro F1=0.8643, стало с новыми параметрами macro F1=8787.

Посмотрим на вклад различных признаков.

In [241]:
analyze_features(model=best_model_LR)

,target,feature,weight,word,category
0,0,x5018,5.103071,abrash,sci.med
1,0,x29783,4.131556,rumours,sci.med
2,0,x15849,3.833022,fractals,sci.med
3,0,x27100,3.551435,pov,sci.med
4,0,x2983,3.022023,486,sci.med
5,0,x30836,2.890158,shograf,sci.med
6,0,x7989,2.846358,bmp,sci.med
7,0,x2699,2.841036,3do,sci.med
8,0,x6163,2.742146,animation,sci.med
9,0,x18958,2.536434,ineed,sci.med


## Деревья решений

До подбора параметров

In [0]:
model_DT = DecisionTreeClassifier().fit(X_train, y_train)
predicted_DT = model_DT.predict(X_test)

In [189]:
show_macro_f1(y_test, predicted_DT)

macro F1=0.7323


In [190]:
results(model_DT)

result on train: 0.37399150743099785
result on test: 0.27718783216075316


После

In [243]:
DT = DecisionTreeClassifier()
parameters_DT = {'splitter' : ['best', 'random'],
                  'max_depth' : [10, 20, 30, 50, 100]}

make_grid_search(DT, parameters_DT, 'f1_macro', n_jobs=5)

Best score: 0.7913258813478542
Best parameters: {'max_depth': 50, 'splitter': 'random'}


In [0]:
best_model_DT = DecisionTreeClassifier(max_depth=50, splitter='random').fit(X_train, y_train)
predicted_DT = best_model_DT.predict(X_test)

In [245]:
show_macro_f1(y_test, predicted_DT)

macro F1=0.7345


In [193]:
results(best_model_DT)

result on train: 0.3571230845279288
result on test: 0.27605427420936646


Макро f1 было 0.7323, стало 0.7345, то улучшения совсем маленькие. В любом случае посмотрим на вклад признаков.

In [242]:
features = pd.DataFrame(X_train.toarray())
feature_imps = pd.DataFrame()
feature_imps["feature"] = features.columns
feature_imps["importance"] = best_model_DT.feature_importances_
feature_imps[["feature", "importance"]].sort_values(by='importance', ascending=False)[:10]

,feature,importance
31595,31595,0.130002
16895,16895,0.084205
26643,26643,0.047640
12815,12815,0.035789
25295,25295,0.034852
9841,9841,0.031601
13062,13062,0.031318
15356,15356,0.023421
32752,32752,0.022965
27174,27174,0.021674


## Случайный лес

До кросс-валидации

In [0]:
model_RF = RandomForestClassifier().fit(X_train, y_train)
predicted_RF = model_RF.predict(X_test)

In [198]:
show_macro_f1(y_test, predicted_RF)

macro F1=0.8219


In [199]:
results(model_RF)

result on train: 0.37399150743099785
result on test: 0.22641844250891202


После

In [53]:
RF = RandomForestClassifier()
parameters_RF = {'n_estimators' : [10, 20, 30, 50, 100], 'criterion': ['gini', 'entropy'], 'min_samples_leaf': [3, 5, 7, 10],'max_features': ['auto', 'sqrt', 'log2']}

make_grid_search(RF, parameters_RF, 'f1_macro', n_jobs=1)

Best score: 0.8994840462063676
Best parameters: {'criterion': 'gini', 'max_features': 'auto', 'min_samples_leaf': 3, 'n_estimators': 100}


In [0]:
best_model_RF = RandomForestClassifier(criterion='gini', max_features='auto', min_samples_leaf=3, n_estimators=100).fit(X_train, y_train)
predicted_RF = best_model_RF.predict(X_test)

In [201]:
show_macro_f1(y_test, predicted_RF)

macro F1=0.8354


In [202]:
results(best_model_RF)

result on train: 0.2980757256736496
result on test: 0.1685397368411885


До macro f1 было 0.8219, после стало 0.8354, то есть, к сожалению, мы снова не наблюдаем большого улучшения качества. Посмотрим на признаки:

In [246]:
analyze_features(best_model_RF)

,feature,weight,std,word
0,x31595,0.024378,0.044016,space
1,x16895,0.019884,0.035311,graphics
2,x25295,0.014714,0.025689,orbit
3,x24027,0.010067,0.021048,nasa
4,x26643,0.009861,0.022463,pitt
5,x21046,0.006964,0.016391,launch
6,x22678,0.006117,0.013201,medical
7,x16314,0.006112,0.018539,geb
8,x31131,0.006036,0.016936,skepticism
9,x27108,0.005851,0.010288,power


*Используйте параметры CountVectorizer, для того, чтобы уменьшить количество признаков и убрать нерелевантные (например числа, токены слишком низкой или высокой документной частотой, и т.д.), постарайтесь добиться улучшения результатов работы моделей (снова выводите результаты модели на трейне и на тесте, чтобы видеть, уменьшается ли переобучение)*

Попробуем ограничить частотность, добавить биграммы и стоп-слова.

In [0]:
count_vect = CountVectorizer(max_df=200, min_df=3, ngram_range=(1,2), stop_words='english')
y_train = data_train.target
y_test = data_test.target
X_train = count_vect.fit_transform(data_train.data) 
X_test = count_vect.transform(data_test.data) 

In [278]:
%%time
best_model_LR = LogisticRegression(penalty='l1', C=100.0, solver='liblinear', max_iter=100).fit(X_train, y_train)
predicted_LR = best_model_LR.predict(X_test)

CPU times: user 28.4 s, sys: 8.85 ms, total: 28.5 s
Wall time: 28.5 s


In [279]:
show_macro_f1(y_test, predicted_LR)

macro F1=0.8652


In [280]:
results(best_model_LR)

result on train: 0.37399150743099785
result on test: 0.32881675082684036


In [0]:
best_model_DT = DecisionTreeClassifier(max_depth=50, splitter='random').fit(X_train, y_train)
predicted_DT = best_model_DT.predict(X_test)

In [282]:
show_macro_f1(y_test, predicted_DT)

macro F1=0.6932


In [283]:
results(best_model_DT)

result on train: 0.3285428858677131
result on test: 0.2705484733216819


In [0]:
best_model_RF = RandomForestClassifier(criterion='gini', max_features='auto', min_samples_leaf=3, n_estimators=100).fit(X_train, y_train)
predicted_RF = best_model_RF.predict(X_test)

In [285]:
show_macro_f1(y_test, predicted_RF)

macro F1=0.8711


In [286]:
results(best_model_RF)

result on train: 0.29503204468132427
result on test: 0.16752626485394972


У деревьев решений F1 мера немного ухудшилась: было 0.7334, стало 0.6932. То же можно наблюдать у логрегрессии: ее F1 мера стала 0.8652, хотя была 0.8775. Случайный лес, наоборот, показал прирост: мера была 0.8354, стала 0.8711. При этом при изменении разных параметров CountVectorizer я неизменно получала более низкий результат для логрегрессии и деревьев решений, поэтому можно сделать вывод, что эти два классификатора работают лучше, когда параметры векторайзера не трогают.